# SUSA CX Kaggle Capstone Project
## Part 3: Hyperparameter Tuning, Decision Trees, Ensemble Learning

### Table Of Contents
* [Introduction](#section1)
* [Hyperparameters](#section2)
* [GridSearch](#section2i)
* [Decision Trees](#section3)   
* [Random Forest](#section4)
* [Ensemble learning](#section5)
* [Conclusion](#conclusion)
* [Additional Reading](#reading)


### Hosted by and maintained by the [Statistics Undergraduate Students Association (SUSA)](https://susa.berkeley.edu). Originally authored by [Patrick Chao](mailto:prc@berkeley.edu) & [Noah Gundotra](mailto:noah.gundotra@berkeley.edu).

<a id='section1'></a>
# SUSA CX Kaggle Capstone Project

Welcome to the second week of the SUSA CX Kaggle Capstone! Now that you have an understanding of your data, a cleaned dataset, and a working model, now it's time to improve your model performance with some new tricks and techniques. Now that we have explored regularization techniques, PCA for feature selection and dimension reduction, and feature engineering, we will delve into more technical details and advanced models.



## Logistics

Most of the logistics are the same as last week, but we are repeating them here for your convenience. Please let us know if you or your teammates are feeling nervous about the pace of this project - remember that we are not grading you on your project, and we really try to make the notebooks relatively easy and fast to code through. If for any reason you are feeling overwhelmed or frustrated, please DM us or talk to us in person. We want all of you to have a productive, healthy, and fun time learning data science! If you have any suggestions or recommendations on how to improve, please do not hesitate to reach out!

## Datathon

For those of you that came to the Datathon, we hope you all had a great time! It seemed that many people got a lot of work done and learned a ton of new materials. We had a quick lecture by Patrick on the mathematical intuitions on regularization and PCA. If you couldn't make it or didn't understand all the details, feel free to reach out! Also if you felt that the lecture was very helpful and you would like to see more similar stuff, let us know!

### Mandatory Office Hours

Because this is such a large project, you and your team will surely have to work on it outside of meetings. In order to get you guys to seek help from this project, we are making it **mandatory** for you and your group to attend **two (2)** SUSA Office Hours over the next 4 weeks. This will allow questions to be answered outside of the regular meetings and will help promote collaboration with more experienced SUSA members.

The schedule of SUSA office hours are below:
https://susa.berkeley.edu/calendar#officehours-table

We understand that most of you will end up going to Arun or Patrick's office hours, but we highly encourage you to go to other people's office hours as well. There are many qualified SUSA mentors who can help and this could be an opportunity for you to meet them.

# Hyperparameters and Tuning

As we have discussed in the project workflow, we are beginning to investigate more technical models. However, as we continue to explore more and more complex models, we may lose interpretability. Many problems in machine learning or data science eventually reduce to some parameters we are incapable of understanding or predicting ahead of time. These may be parameters inherent to the dataset or based on the model.

If we look back at k-Nearest Neighbors, based on different values of $k$ we obtain various models. For $k=1$, we obtain a low bias and high variance estimate. Our estimate is highly dependent on the data available. As $k$ increases, our bias increases but the variance decreases. We would like to minimize both bias and variance, thus the optimal value of $k$ lies somewhere between these values. How do we determine this value? Unfortunately there is no golden bullet or magical formula we may calculate to answer this question.

These mystical quantities we would like to evaluate are known as **hyperparameters**. Hyperparameters are values in our model that we cannot optimize directly, instead we may have to explore a variety of examples before determining the optimal value. In kNN, the value of $k$ characterizes the model. Thus we may describe this as a **model hyperparameter**. If we were using polynomial regression, the degree of the model would also be a model hyperparameter, as we can arbitrarily select the power and it characterizes our model.

Another example of a hyperparameter is the $\lambda$ used in Ridge and LASSO regression. There is essentially no method of evaluating the optimal value of $\lambda$ through calculations, instead we must experiment and emperically determine the best value. These do not determine the form of model, but affect the optimization. Thus this is an **optimization hyperparameter**.

We may imagine hyperparameters as a bunch of individual knobs we may turn. Consider that we are visiting our friend and staying at her place. However, you did not realize that she is actually an alien and her house is filled with very strange objects. When you head to bed, you attempt to use her shower, but see that her shower is has a dozen of knobs that control the temperature of the water coming out! We only have a single output to work off of, but many different knobs or *parameters* to adjust. If the water is too hot, we can turn random knobs until it becomes cold, and learn a bit about our environment. We may determine that some knobs are more or less sensitive, just like like hyperparameters. Each knob in the shower is equivalent to a hyperparameter we can tune in a model.

# Grid Search

Consider we have two hyperparameters $A$ and $B$ that range from $0$ to $100$. 

> 1. Discuss with a group on possible methods of deciding the optimal value of $A$ and $B$?
> 2. Is it possible to find the exact best values of $A$ and $B$?

One of the most naive approaches is to simply create a grid of values for $A$ and $B$, hence the name grid search. We could consider choosing the values where $A$ or $B$ are equal to $0,50,100$, and determine the validation error on all pairs of hyperparameters. Since there are $3$ values for both $A$ and $B$, there are $9$ total possible choices for $(A,B)$.

Consider the following grid below.

<img src="GRAPHICS/grid1.png" width="30%">

The grid represents the validation error for all $(A,B)$ from $[0,50,100]$. It may seem that $(A,B)=(50,100)$ is the best value with an error of $7$, but our scale is quite coarse. We could instead have four values for $A$ and $B$, resulting in $16$ total calculations. This is shown below with $[0,33,66,100]$.
<img src="GRAPHICS/grid2.png" width="30%">

Now the optimal value shifts to $(A,B)=(66,33)$ with an error of $7$, but this will change again as our interval becomes more and more fine. However, a few issues immediately arise. First, the number of calculations increases quadratically with the number of values for each interval! This is quite bad, as quadratic functions grow quite quickly. Secondly, we can never really determine the exact optimal value unless we had infinitely fine intervals. Lastly, we only had an decrease in error from $7$ to $5$ with $9$ to $16$ calculations. Is this actually worthwhile? Sometimes performing more searches does not actually help. These are all various tradeoffs to decide, at a certain point the number of calculations is not worth the added benefit of well tuned parameters.

Now, this is only for the two variable case with $A$ and $B$. In practice, models may have many variables, even dozens! If we consider $10$ parameters, just doing two values for an interval results in $2^10=1024$ calculations! Keep in mind that each value in the grid requires a separate model to be trained using the parameters, which results in $1024$ different models to be trained with just a comparison of two values for each parameter! The number of models trained is equal to $m^n$ where $m$ is the number of values in the interval, and $n$ is the number of hyperparameters.

Grid search can be very challenging due to this issue, and suffers from the curse of dimensionality. There are other approaches that are used in practice. Instead of methodically selecting all pairs of values in the cartesian product, we could perform a [random search](https://en.wikipedia.org/wiki/Random_search) through parameters. Even more complex, we can create a model to estimate the optimal parameters. This leads into the field of [metalearning](, learning to learn.

# Decision Trees

# Random Forest

# Ensemble Learning

# Conclusion

# Additional Reading